In [1]:
import pandas as pd
from datetime import date, timedelta
from modifiedGlicko import Team, ratingUpdate
# import glicko2

[1025.4859444495833, 1043.715823828581, 1047.824089951749, 1051.5941058924072, 1061.9907353813232, 1065.1521202133881, 1077.1186318584948, 1091.1026861482874]
[1015.291566641589, 1010.7035561822365, 1006.5859913275391, 995.3000128213282, 1005.6526056392763, 1005.614515951363, 1020.5106602053686, 1006.5437109692593]
[1005.0926139761883, 1041.4948204395723, 1037.3863900415756, 1018.5504986576087, 994.3211575463804, 981.5249091126782, 966.6340652695692, 952.725441871738]
[984.708433358411, 966.4785540053537, 974.7183971192803, 1001.0827651280124, 1007.9672927713647, 995.1990911116804, 1007.1292367921378, 1021.1005828304977]
[974.5140555504167, 938.0213918889998, 929.7819643366753, 948.6814223988326, 945.2814947901627, 938.8937503740217, 929.9429834612512, 929.9760904691803]


In [2]:
df = pd.read_csv("bucs_results.csv")

postcodes = pd.read_csv("institute_postcodes.csv")

# print the first few rows of the DataFrame to verify that it was loaded correctly
df.head()
df = df.drop(columns=['No Score Type', 'Walkover Reason','Knockout Fixtures', 'Number of Walkovers', 'Voids', 'Blowout Game(s)', 'Zero Score?', 'Blowout to Zero', 'Team Conceding Walkover', 'Fixtures',
       'League Fixtures', 'Time'])

In [3]:
df = df[df["Fixture type"]=="League"]

In [4]:
def generate_glicko_init_rating(comp):
    if "(National)" in comp:
        return 2000
    if "North/South" in comp:
        return 2000
    if "Tier 1" in comp:
        return 1800
    if "Tier 2" in comp:
        return 1400
    if "Tier 3" in comp:
        return 1200
    if "Tier 4" in comp:
        return 1000
    if "Tier 5" in comp:
        return 800

In [5]:
df['HG_RANK_Initial'] = df.apply(lambda row: generate_glicko_init_rating(row.Competition), axis=1)
df['AG_RANK_Initial'] = df.apply(lambda row: generate_glicko_init_rating(row.Competition), axis=1)

In [6]:
df

,Community,Date,Fixture type,Competition,Home,Home Score,Away Score,Away,Winner,Venue,Team Type,Conference,Tier,Score Margin,Total Points,HG_RANK_Initial,AG_RANK_Initial
1,BUCS Hockey 19-20,25/09/2019,League,Men's Scottish Tier 3,Heriot-Watt Men's 2,0,0,Edinburgh Men's 5,NaN,Peffermill Playing Fields Provider: BUCS Hocke...,Men's,Scottish,Season,0.0,0.0,1200.0,1200.0
2,BUCS Hockey 19-20,25/09/2019,League,Men's Scottish Tier 4,Aberdeen Men's 3,0,1,Stirling Men's 3,Stirling Men's 3,"Hillhead Pitches, Hillhead Centre Provider: BU...",Men's,Scottish,Season,1.0,1.0,1000.0,1000.0
3,BUCS Hockey 19-20,25/09/2019,League,ICG Women's Scottish Tier 5,Edinburgh Women's 7,4,2,Heriot-Watt Women's 2,Edinburgh Women's 7,"Sport & Exercise Peffermill, Water Based 1 Pro...",Women's,Scottish,Season,2.0,6.0,800.0,800.0
4,BUCS Hockey 19-20,25/09/2019,League,Men's Scottish Tier 1,Aberdeen Men's 1,3,2,Heriot-Watt Men's 1,Aberdeen Men's 1,ASV Hockey Pitch (Water based) Provider: BUCS ...,Men's,Scottish,Tier 1,1.0,5.0,1800.0,1800.0
5,BUCS Hockey 19-20,25/09/2019,League,ICG Women's Scottish Tier 1,Glasgow Women's 1,0,1,Edinburgh Women's 2,Edinburgh Women's 2,"Garscube Sports Complex, West of Scotland Scie...",Women's,Scottish,Tier 1,1.0,1.0,1800.0,1800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7830,BUCS Hockey 22-23,05/10/2022,League,ICG Women's Scottish Tier 5,Abertay Women's 2,0,7,St Andrews Women's 6,St Andrews Women's 6,"DISCMains Loan, Dundee, DD4 7AA Provider: Hock...",Women's,Scottish,Season,7.0,7.0,800.0,800.0
7831,BUCS Hockey 22-23,05/10/2022,League,ICG Women's Scottish Tier 2,Heriot-Watt Women's 1,1,2,Edinburgh Women's 4,Edinburgh Women's 4,"Sport & Exercise Peffermill, Water Based 1 Pro...",Women's,Scottish,Tier 2,1.0,3.0,1400.0,1400.0
7832,BUCS Hockey 22-23,05/10/2022,League,ICG Women's Scottish Tier 2,Robert Gordon Women's 1,4,4,Aberdeen Women's 1,NaN,Countesswells Sports Field Provider: RGC,Women's,Scottish,Tier 2,0.0,8.0,1400.0,1400.0
7833,BUCS Hockey 22-23,05/10/2022,League,ICG Women's Scottish Tier 3,Glasgow Women's 3,2,0,Strathclyde Women's 2,Glasgow Women's 3,"Garscube Sports Complex, West of Scotland Scie...",Women's,Scottish,Season,2.0,2.0,1200.0,1200.0


In [14]:
team_gscores = {}

for i, team in df[df["Community"]=="BUCS Hockey 22-23"].iterrows():
    team_gscores[team.Home] = team.HG_RANK_Initial
    team_gscores[team.Away] = team.AG_RANK_Initial
    
print(team_gscores)

team_gObj = {}

for team, gscore in team_gscores.items():
    team_gObj[team] = Team(team, gscore, initial_deviation=150, initial_volatility=0.001)
    
# del team_gscores 

{"Cardiff Met Women's 2": 1800.0, "Cardiff Women's 2": 1800.0, "Newcastle Women's 1": 2000.0, "Loughborough Women's 2": 2000.0, "Glasgow Women's 1": 2000.0, "Nottingham Women's 2": 2000.0, "UCL Men's 5": 1000.0, "UCL Men's 4 (RUMS)": 1000.0, "Edinburgh Men's 1": 2000.0, "Nottingham Men's 1": 2000.0, "Edinburgh Women's 1": 2000.0, "Nottingham Women's 1": 2000.0, "Birmingham Men's 6": 1000.0, "Nottingham Men's 7": 1000.0, "Nottingham Trent Women's 3": 1400.0, "Oxford Brookes Women's 2": 1400.0, "Heriot-Watt Men's 2": 1200.0, "St Andrews Men's 2": 1200.0, "Liverpool Women's 2": 1400.0, "Liverpool John Moores Women's 1": 1400.0, "Edinburgh Napier Men's 1": 1200.0, "Abertay Men's 1": 1200.0, "Imperial Women's 4 (Medics)": 800.0, "St George's Women's 2": 800.0, "Arts London Women's 1": 1200.0, "St George's Women's 1": 1200.0, "Bolton Men's 1": 800.0, "Wrexham Glyndwr Men's 1": 800.0, "City Women's 1": 1000.0, "King's College Women's 5 (GKT)": 1000.0, "Nottingham Trent Women's 1": 2000.0, "Du

In [8]:
for i, row in df.iterrows():
    home_gobj = team_gObj.get(row['Home'], Team(row['Home'], row['HG_RANK_Initial']))
    away_gobj = team_gObj.get(row['Away'], Team(row['Away'], row['AG_RANK_Initial']))
    score_diff = int(row['Home Score']) - int(row['Away Score'])
    ratingUpdate(
        team_gObj.get(row['Home'], Team(row['Home'], row['HG_RANK_Initial'])), 
        team_gObj.get(row['Away'], Team(row['Away'], row['AG_RANK_Initial'])),
        score_diff
    )

print(team_gObj)

{"Cardiff Met Women's 2": Cardiff Met Women's 2, "Cardiff Women's 2": Cardiff Women's 2, "Newcastle Women's 1": Newcastle Women's 1, "Loughborough Women's 2": Loughborough Women's 2, "Glasgow Women's 1": Glasgow Women's 1, "Nottingham Women's 2": Nottingham Women's 2, "UCL Men's 5": UCL Men's 5, "UCL Men's 4 (RUMS)": UCL Men's 4 (RUMS), "Edinburgh Men's 1": Edinburgh Men's 1, "Nottingham Men's 1": Nottingham Men's 1, "Edinburgh Women's 1": Edinburgh Women's 1, "Nottingham Women's 1": Nottingham Women's 1, "Birmingham Men's 6": Birmingham Men's 6, "Nottingham Men's 7": Nottingham Men's 7, "Nottingham Trent Women's 3": Nottingham Trent Women's 3, "Oxford Brookes Women's 2": Oxford Brookes Women's 2, "Heriot-Watt Men's 2": Heriot-Watt Men's 2, "St Andrews Men's 2": St Andrews Men's 2, "Liverpool Women's 2": Liverpool Women's 2, "Liverpool John Moores Women's 1": Liverpool John Moores Women's 1, "Edinburgh Napier Men's 1": Edinburgh Napier Men's 1, "Abertay Men's 1": Abertay Men's 1, "Impe

In [9]:
for team, obj in team_gObj.items():
    if "Women's" in team:
        continue
    if "Nottingham" not in team:
        continue
    print(team, obj.rating, obj.RD)

Nottingham Men's 1 100 nan
Nottingham Men's 7 1032.6869289109504 106.01923922217873
Nottingham Trent Men's 2 100 nan
Nottingham Men's 2 100 nan
Nottingham Men's 6 1337.3143699664554 81.38046908895313
Nottingham Men's 5 1507.5398582021098 75.20171382960262
Nottingham Trent Men's 1 100 nan
Nottingham Trent Men's 6 1062.413671088599 115.24910700284158
Nottingham Trent Men's 4 1907.123478350797 61.574997024102416
Nottingham Trent Men's 3 1869.6484835467975 62.638785042467
Nottingham Men's 4 1797.7100598050263 75.31898453081871
Nottingham Men's 3 1836.9879255307137 75.31142525578386
Nottingham Trent Men's 5 1268.6930709336725 85.9238222895035
